<a href="https://colab.research.google.com/github/luigiselmi/dl_tensorflow/blob/main/convnet_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Deep Learning for Computer Vision - ConvNet Visualization

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [2]:
model = keras.models.load_model("models/convnet_from_scratch_with_augmentation.keras")
model.summary()

OSError: No file or directory found at models/convnet_from_scratch_with_augmentation.keras